In [2]:
# encoding=utf-8
import os.path as osp
import os
import copy
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.data import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.datasets import ICEWS18
from torch_geometric.nn import TGNMemory, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models.tgn import (LastNeighborLoader, IdentityMessage, MeanAggregator,
                                           LastAggregator)
from torch_geometric import *
from torch_geometric.utils import negative_sampling
from tqdm import tqdm
import networkx as nx
import numpy as np
import math
import copy
import re
import time
import json
import pandas as pd
from random import choice
import gc

# We conducted this experiment on CPU, including training, loading and testing models.
# For reproducibility, we recommend users to load and test our models on CPU.
device = 'cpu'
# msg structure:    [src_node_feature,edge_attr,dst_node_feature]

# compute the best partition 
import datetime
# import community as community_louvain

import xxhash
# Find the edge index which the edge vector is corresponding to
def tensor_find(t,x):
    t_np=t.cpu().numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1


def std(t):
    t = np.array(t)
    return np.std(t)


def var(t):
    t = np.array(t)
    return np.var(t)


def mean(t):
    t = np.array(t)
    return np.mean(t)

def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()


def cal_pos_edges_loss(link_pred_ratio):
    loss=[]
    for i in link_pred_ratio:
        loss.append(criterion(i,torch.ones(1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_multiclass(link_pred_ratio,labels):
    loss=[] 
    for i in range(len(link_pred_ratio)):
        loss.append(criterion(link_pred_ratio[i].reshape(1,-1),labels[i].reshape(-1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_autoencoder(decoded,msg):
    loss=[] 
    for i in range(len(decoded)):
        loss.append(criterion(decoded[i].reshape(1,-1),msg[i].reshape(-1)))
    return torch.tensor(loss)

In [3]:
%autosave 120  

Autosaving every 120 seconds


In [4]:
from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def ns_time_to_datetime_US(ns):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int nano timestamp
    :return: datetime   format: 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')

    return s

def datetime_to_ns_time(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp = timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str   format: %Y-%m-%d %H:%M:%S   e.g. 2013-10-10 23:40:00
    :return: nano timestamp
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp = timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)

In [4]:
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_e5_clearscope_dataset_db',
                           host = '/var/run/postgresql/',
                           user = 'postgres',
                           password = 'postgres',
                           port = '5432'
                          )

cur = connect.cursor()

In [5]:
graph_5_8=torch.load("./train_graphs/graph_5_8.TemporalData.simple").to(device=device)
graph_5_9=torch.load("./train_graphs/graph_5_9.TemporalData.simple").to(device=device)
graph_5_11=torch.load("./train_graphs/graph_5_11.TemporalData.simple").to(device=device)


train_data=graph_5_8

In [6]:
# Constructing the map for nodeid to msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # nodeid => msg and node hash => nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  

In [7]:
rel2id={1: 'EVENT_ACCEPT',
 'EVENT_ACCEPT': 1,
 2: 'EVENT_CLONE',
 'EVENT_CLONE': 2,
 3: 'EVENT_CLOSE',
 'EVENT_CLOSE': 3,
 4: 'EVENT_CREATE_OBJECT',
 'EVENT_CREATE_OBJECT': 4,
 5: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 5,
 6: 'EVENT_OPEN',
 'EVENT_OPEN': 6,
 7: 'EVENT_READ',
 'EVENT_READ': 7,
 8: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 8,
 9: 'EVENT_SENDTO',
 'EVENT_SENDTO': 9,
 10: 'EVENT_WRITE',
 'EVENT_WRITE': 10}

In [8]:
# train_data, val_data, test_data = data.train_val_test_split(val_ratio=0.15, test_ratio=0.15)
# max_node_num = max(torch.cat([data.dst,data.src]))+1
# max_node_num = data.num_nodes+1
max_node_num = 139961  # +1
# min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())
min_dst_idx, max_dst_idx = 0, max_node_num
neighbor_loader = LastNeighborLoader(max_node_num, size=20, device=device)

In [9]:
class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super(GraphAttentionEmbedding, self).__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels, heads=8,
                                    dropout=0.0, edge_dim=edge_dim)
        self.conv2 = TransformerConv(in_channels*8, out_channels,heads=1, concat=False,
                             dropout=0.0, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        last_update.to(device)
        x = x.to(device)
        t = t.to(device)
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        x = F.relu(self.conv(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin_src = Linear(in_channels, in_channels*2)
        self.lin_dst = Linear(in_channels, in_channels*2)
       
        self.lin_seq = nn.Sequential(
            Linear(in_channels * 4, in_channels * 8),
            torch.nn.BatchNorm1d(in_channels * 8),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels * 8, in_channels * 2),
            torch.nn.BatchNorm1d(in_channels * 2),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels * 2, int(in_channels // 2)),
            torch.nn.BatchNorm1d(int(in_channels // 2)),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(int(in_channels // 2), train_data.msg.shape[1] - 32)
        )

    def forward(self, z_src, z_dst):
        h = torch.cat([self.lin_src(z_src) , self.lin_dst(z_dst)],dim=-1)      
         
        h = self.lin_seq (h)
        
        return h
        



memory_dim = 100         # node state
time_dim = 100
embedding_dim = 100      # edge embedding

memory = TGNMemory(
    max_node_num,
    train_data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(train_data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=train_data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.00005, eps=1e-08,weight_decay=0.01)


# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(max_node_num, dtype=torch.long, device=device)

saved_nodes=set()

In [10]:
BATCH=1024
def train(train_data):

    
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.
    saved_nodes=set()

    total_loss = 0
    
#     print("train_before_stage_data:",train_data)
    for batch in train_data.seq_batches(batch_size=BATCH):
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg        
        
        n_id = torch.cat([src, pos_dst]).unique()
#         n_id = torch.cat([src, pos_dst, neg_src, neg_dst]).unique()
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
      
        z = gnn(z, last_update, edge_index, train_data.t[e_id], train_data.msg[e_id])
        
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])       

        y_pred = torch.cat([pos_out], dim=0)
        
#         y_true = torch.cat([torch.zeros(pos_out.size(0),1),torch.ones(neg_out.size(0),1)], dim=0)
        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l)           
          
        y_true = torch.tensor(y_true).to(device=device)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)
        
        loss = criterion(y_pred, y_true)
        
#         loss = criterion(pos_out, torch.ones_like(pos_out))
#         loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
#         for i in range(len(src)):
#             saved_nodes.add(int(src[i]))
#             saved_nodes.add(int(pos_dst[i]))

        loss.backward()
        optimizer.step()
        memory.detach()
#         print(z.shape)
        total_loss += float(loss) * batch.num_events
#     print("trained_stage_data:",train_data)
    return total_loss / train_data.num_events





In [ ]:
train_graphs=[graph_5_8, graph_5_9, graph_5_11]

for epoch in tqdm(range(1, 31)):
    for g in train_graphs:
        loss = train(g)
        print(f'  Epoch: {epoch:02d}, Loss: {loss:.4f}')
#     scheduler.step()
model=[memory,gnn, link_pred,neighbor_loader]
os.system("mkdir -p ./models/")
torch.save(model,"./models/models.pt")

# Test

In [11]:
import time 

@torch.no_grad()
def test_day_new(inference_data,path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
    memory.eval()
    gnn.eval()
    link_pred.eval()
    
    memory.reset_state()  # Start with a fresh memory. 
    neighbor_loader.reset_state()  # Start with an empty graph.
    
    time_with_loss={}
    total_loss = 0    
    edge_list=[]
    
    unique_nodes=torch.tensor([]).to(device=device)
    total_edges=0


    start_time=inference_data.t[0]
    event_count=0
    
    pos_o=[]
    
    loss_list=[]
    

    print("after merge:",inference_data)
    
    # Record the running time to evaluate the performance
    start = time.perf_counter()

    for batch in inference_data.seq_batches(batch_size=BATCH):
        
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        unique_nodes=torch.cat([unique_nodes,src,pos_dst]).unique()
        total_edges+=BATCH
        
       
        n_id = torch.cat([src, pos_dst]).unique()       
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, inference_data.t[e_id], inference_data.msg[e_id])

        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])
        
        pos_o.append(pos_out)
        y_pred = torch.cat([pos_out], dim=0)
#         y_true = torch.cat(
#             [torch.ones(pos_out.size(0))], dim=0).to(torch.long)     
#         y_true=y_true.reshape(-1).to(torch.long)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l) 
        y_true = torch.tensor(y_true).to(device=device)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)

        # Only consider which edge hasn't been correctly predicted.
        # For benign graphs, the behaviors patterns are similar and therefore their losses are small
        # For anoamlous behaviors, some behaviors might not be seen before, so the probability of predicting those edges are low. Thus their losses are high.
        loss = criterion(y_pred, y_true)

        total_loss += float(loss) * batch.num_events
     
        
        # update the edges in the batch to the memory and neighbor_loader
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
        # compute the loss for each edge
        each_edge_loss= cal_pos_edges_loss_multiclass(pos_out,y_true)
        
        for i in range(len(pos_out)):
            srcnode=int(src[i])
            dstnode=int(pos_dst[i])  
            
            srcmsg=str(nodeid2msg[srcnode]) 
            dstmsg=str(nodeid2msg[dstnode])
            t_var=int(t[i])
            edgeindex=tensor_find(msg[i][16:-16],1)   
            edge_type=rel2id[edgeindex]
            loss=each_edge_loss[i]    

            temp_dic={}
            temp_dic['loss']=float(loss)
            temp_dic['srcnode']=srcnode
            temp_dic['dstnode']=dstnode
            temp_dic['srcmsg']=srcmsg
            temp_dic['dstmsg']=dstmsg
            temp_dic['edge_type']=edge_type
            temp_dic['time']=t_var
            
#             if "netflow" in srcmsg or "netflow" in dstmsg:
#                 temp_dic['loss']=0
            edge_list.append(temp_dic)
        
        event_count+=len(batch.src)
        if t[-1]>start_time+60000000000*15:
            # Here is a checkpoint, which records all edge losses in the current time window
#             loss=total_loss/event_count
            time_interval=ns_time_to_datetime_US(start_time)+"~"+ns_time_to_datetime_US(t[-1])

            end = time.perf_counter()
            time_with_loss[time_interval]={'loss':loss,
                                
                                          'nodes_count':len(unique_nodes),
                                          'total_edges':total_edges,
                                          'costed_time':(end-start)}
            
            
            log=open(path+"/"+time_interval+".txt",'w')
            
            for e in edge_list: 
#                 temp_key=e['srcmsg']+e['dstmsg']+e['edge_type']
#                 if temp_key in train_edge_set:      
# #                     e['loss']=(e['loss']-train_edge_set[temp_key]) if e['loss']>=train_edge_set[temp_key] else 0  
# #                     e['loss']=abs(e['loss']-train_edge_set[temp_key])
                    
#                     e['modified']=True
#                 else:
#                     e['modified']=False
                loss+=e['loss']

            loss=loss/event_count   
            print(f'Time: {time_interval}, Loss: {loss:.4f}, Nodes_count: {len(unique_nodes)}, Cost Time: {(end-start):.2f}s')
            edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True)   
            for e in edge_list: 
                log.write(str(e))
                log.write("\n") 
            event_count=0
            total_loss=0
            loss=0
            start_time=t[-1]
            log.close()
            edge_list.clear()
            
 
    return time_with_loss





In [12]:
model=torch.load("./models/models.pt", map_location=device)
memory,gnn, link_pred,neighbor_loader=model


In [13]:
graph_5_12=torch.load("./train_graphs/graph_5_12.TemporalData.simple").to(device=device)
graph_5_14=torch.load("./train_graphs/graph_5_14.TemporalData.simple").to(device=device)
graph_5_15=torch.load("./train_graphs/graph_5_15.TemporalData.simple").to(device=device)
graph_5_17=torch.load("./train_graphs/graph_5_17.TemporalData.simple").to(device=device)

In [14]:
ans_5_8=test_day_new(graph_5_8,"graph_5_8")

after merge: TemporalData(dst=[6060013], msg=[6060013, 42], src=[6060013], t=[6060013])


/home/yinyuanl/anaconda3/envs/kairos/lib/python3.9/site-packages/torch_geometric/nn/conv/transformer_conv.py:211: UserWarning: operator() profile_node %28 : int[] = prim::profile_ivalue(%size.4)
 does not have profile information (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/torch/csrc/jit/codegen/cuda/graph_fuser.cpp:105.)
  alpha = softmax(alpha, index, ptr, size_i)


Time: 2019-05-08 00:00:00.152000000~2019-05-08 00:16:37.159000000, Loss: 0.7120, Nodes_count: 1027, Cost Time: 7.89s
Time: 2019-05-08 00:16:37.159000000~2019-05-08 00:32:09.988000000, Loss: 0.5363, Nodes_count: 1137, Cost Time: 21.50s
Time: 2019-05-08 00:32:09.988000000~2019-05-08 00:47:11.359000000, Loss: 0.5497, Nodes_count: 1192, Cost Time: 26.66s
Time: 2019-05-08 00:47:11.359000000~2019-05-08 01:02:12.559000000, Loss: 0.4982, Nodes_count: 1247, Cost Time: 42.39s
Time: 2019-05-08 01:02:12.559000000~2019-05-08 01:17:13.977000000, Loss: 0.5899, Nodes_count: 1300, Cost Time: 47.65s
Time: 2019-05-08 01:17:13.977000000~2019-05-08 01:32:15.057000000, Loss: 0.5042, Nodes_count: 1367, Cost Time: 63.72s
Time: 2019-05-08 01:32:15.057000000~2019-05-08 01:47:16.557000000, Loss: 0.5488, Nodes_count: 1436, Cost Time: 68.68s
Time: 2019-05-08 01:47:16.557000000~2019-05-08 02:02:17.924000000, Loss: 0.5127, Nodes_count: 1505, Cost Time: 85.88s
Time: 2019-05-08 02:02:17.924000000~2019-05-08 02:17:19.3

Time: 2019-05-08 19:16:56.840000000~2019-05-08 19:32:18.680000000, Loss: 0.5538, Nodes_count: 22079, Cost Time: 1422.78s
Time: 2019-05-08 19:32:18.680000000~2019-05-08 19:47:22.439000000, Loss: 0.4772, Nodes_count: 22133, Cost Time: 1434.76s
Time: 2019-05-08 19:47:22.439000000~2019-05-08 20:03:20.623000000, Loss: 0.5031, Nodes_count: 22224, Cost Time: 1439.56s
Time: 2019-05-08 20:03:20.623000000~2019-05-08 20:18:21.783000000, Loss: 0.4716, Nodes_count: 22271, Cost Time: 1453.34s
Time: 2019-05-08 20:18:21.783000000~2019-05-08 20:33:23.285000000, Loss: 0.4788, Nodes_count: 22319, Cost Time: 1458.59s
Time: 2019-05-08 20:33:23.285000000~2019-05-08 20:48:24.371000000, Loss: 0.4729, Nodes_count: 22359, Cost Time: 1472.77s
Time: 2019-05-08 20:48:24.371000000~2019-05-08 21:03:25.813000000, Loss: 0.4733, Nodes_count: 22405, Cost Time: 1477.88s
Time: 2019-05-08 21:03:25.813000000~2019-05-08 21:18:27.042000000, Loss: 0.4760, Nodes_count: 22455, Cost Time: 1491.22s
Time: 2019-05-08 21:18:27.042000

In [15]:
ans_5_9=test_day_new(graph_5_9,"graph_5_9")

after merge: TemporalData(dst=[8438455], msg=[8438455, 42], src=[8438455], t=[8438455])
Time: 2019-05-09 00:00:00.288000000~2019-05-09 00:15:17.459000000, Loss: 0.5977, Nodes_count: 770, Cost Time: 8.28s
Time: 2019-05-09 00:15:17.459000000~2019-05-09 00:32:11.493000000, Loss: 0.5366, Nodes_count: 866, Cost Time: 13.16s
Time: 2019-05-09 00:32:11.493000000~2019-05-09 00:47:26.637000000, Loss: 0.4846, Nodes_count: 918, Cost Time: 26.56s
Time: 2019-05-09 00:47:26.637000000~2019-05-09 01:02:46.300000000, Loss: 0.4879, Nodes_count: 972, Cost Time: 31.10s
Time: 2019-05-09 01:02:46.300000000~2019-05-09 01:18:00.466000000, Loss: 0.4838, Nodes_count: 1023, Cost Time: 45.92s
Time: 2019-05-09 01:18:00.466000000~2019-05-09 01:33:21.459000000, Loss: 0.4953, Nodes_count: 1073, Cost Time: 50.36s
Time: 2019-05-09 01:33:21.459000000~2019-05-09 01:48:44.527000000, Loss: 0.4802, Nodes_count: 1121, Cost Time: 64.78s
Time: 2019-05-09 01:48:44.527000000~2019-05-09 02:03:51.619000000, Loss: 0.5013, Nodes_coun

Time: 2019-05-09 17:05:07.553000000~2019-05-09 17:20:08.838000000, Loss: 0.5278, Nodes_count: 18160, Cost Time: 1656.11s
Time: 2019-05-09 17:20:08.838000000~2019-05-09 17:35:10.111000000, Loss: 0.6546, Nodes_count: 18417, Cost Time: 1677.69s
Time: 2019-05-09 17:35:10.111000000~2019-05-09 17:50:11.084000000, Loss: 0.6550, Nodes_count: 18861, Cost Time: 1730.69s
Time: 2019-05-09 17:50:11.084000000~2019-05-09 18:05:12.449000000, Loss: 0.6631, Nodes_count: 19261, Cost Time: 1763.60s
Time: 2019-05-09 18:05:12.449000000~2019-05-09 18:20:13.563000000, Loss: 0.6323, Nodes_count: 19578, Cost Time: 1810.49s
Time: 2019-05-09 18:20:13.563000000~2019-05-09 18:35:15.440000000, Loss: 0.6724, Nodes_count: 20387, Cost Time: 1874.05s
Time: 2019-05-09 18:35:15.440000000~2019-05-09 18:50:17.094000000, Loss: 0.6344, Nodes_count: 20725, Cost Time: 1914.45s
Time: 2019-05-09 18:50:17.094000000~2019-05-09 19:05:19.380000000, Loss: 0.6668, Nodes_count: 21335, Cost Time: 1966.12s
Time: 2019-05-09 19:05:19.380000

In [16]:
ans_5_11=test_day_new(graph_5_11,"graph_5_11")

after merge: TemporalData(dst=[34775], msg=[34775, 42], src=[34775], t=[34775])
Time: 2019-05-11 00:00:10.521000000~2019-05-11 00:41:33.462000000, Loss: 2.0373, Nodes_count: 34, Cost Time: 0.04s
Time: 2019-05-11 00:41:33.462000000~2019-05-11 01:23:21.609000000, Loss: 1.5618, Nodes_count: 58, Cost Time: 0.11s
Time: 2019-05-11 01:23:21.609000000~2019-05-11 02:04:35.648000000, Loss: 0.6643, Nodes_count: 85, Cost Time: 0.17s
Time: 2019-05-11 02:04:35.648000000~2019-05-11 02:46:01.387000000, Loss: 0.5496, Nodes_count: 119, Cost Time: 0.24s
Time: 2019-05-11 02:46:01.387000000~2019-05-11 03:28:01.426000000, Loss: 0.5156, Nodes_count: 140, Cost Time: 0.30s
Time: 2019-05-11 03:28:01.426000000~2019-05-11 04:09:37.675000000, Loss: 0.5406, Nodes_count: 164, Cost Time: 0.36s
Time: 2019-05-11 04:09:37.675000000~2019-05-11 04:50:41.283000000, Loss: 0.5299, Nodes_count: 202, Cost Time: 0.43s
Time: 2019-05-11 04:50:41.283000000~2019-05-11 05:32:05.364000000, Loss: 0.5387, Nodes_count: 233, Cost Time: 0

In [17]:
ans_5_12=test_day_new(graph_5_11,"graph_5_12")

after merge: TemporalData(dst=[34775], msg=[34775, 42], src=[34775], t=[34775])
Time: 2019-05-11 00:00:10.521000000~2019-05-11 00:41:33.462000000, Loss: 2.0373, Nodes_count: 34, Cost Time: 0.05s
Time: 2019-05-11 00:41:33.462000000~2019-05-11 01:23:21.609000000, Loss: 1.5618, Nodes_count: 58, Cost Time: 0.12s
Time: 2019-05-11 01:23:21.609000000~2019-05-11 02:04:35.648000000, Loss: 0.6643, Nodes_count: 85, Cost Time: 0.19s
Time: 2019-05-11 02:04:35.648000000~2019-05-11 02:46:01.387000000, Loss: 0.5496, Nodes_count: 119, Cost Time: 0.26s
Time: 2019-05-11 02:46:01.387000000~2019-05-11 03:28:01.426000000, Loss: 0.5156, Nodes_count: 140, Cost Time: 0.32s
Time: 2019-05-11 03:28:01.426000000~2019-05-11 04:09:37.675000000, Loss: 0.5406, Nodes_count: 164, Cost Time: 0.39s
Time: 2019-05-11 04:09:37.675000000~2019-05-11 04:50:41.283000000, Loss: 0.5299, Nodes_count: 202, Cost Time: 0.48s
Time: 2019-05-11 04:50:41.283000000~2019-05-11 05:32:05.364000000, Loss: 0.5387, Nodes_count: 233, Cost Time: 0

In [18]:
graph_5_14.to(device=device)
ans_5_14=test_day_new(graph_5_14,"graph_5_14")

after merge: TemporalData(dst=[9518075], msg=[9518075, 42], src=[9518075], t=[9518075])
Time: 2019-05-14 00:00:00.110000000~2019-05-14 00:16:55.164000000, Loss: 0.5621, Nodes_count: 831, Cost Time: 19.18s
Time: 2019-05-14 00:16:55.164000000~2019-05-14 00:31:56.512000000, Loss: 0.6636, Nodes_count: 902, Cost Time: 24.05s
Time: 2019-05-14 00:31:56.512000000~2019-05-14 00:46:57.832000000, Loss: 0.4942, Nodes_count: 963, Cost Time: 47.27s
Time: 2019-05-14 00:46:57.832000000~2019-05-14 01:01:59.206000000, Loss: 0.6481, Nodes_count: 1019, Cost Time: 51.91s
Time: 2019-05-14 01:01:59.206000000~2019-05-14 01:17:00.513000000, Loss: 0.4883, Nodes_count: 1089, Cost Time: 75.51s
Time: 2019-05-14 01:17:00.513000000~2019-05-14 01:32:01.927000000, Loss: 0.5433, Nodes_count: 1151, Cost Time: 80.95s
Time: 2019-05-14 01:32:01.927000000~2019-05-14 01:47:02.953000000, Loss: 0.4940, Nodes_count: 1218, Cost Time: 103.32s
Time: 2019-05-14 01:47:02.953000000~2019-05-14 02:02:04.367000000, Loss: 0.5369, Nodes_c

Time: 2019-05-14 17:05:10.964000000~2019-05-14 17:20:12.227000000, Loss: 0.6217, Nodes_count: 24522, Cost Time: 2080.32s
Time: 2019-05-14 17:20:12.227000000~2019-05-14 17:35:13.719000000, Loss: 0.6864, Nodes_count: 24651, Cost Time: 2087.85s
Time: 2019-05-14 17:35:13.719000000~2019-05-14 17:50:26.379000000, Loss: 0.6389, Nodes_count: 25303, Cost Time: 2152.99s
Time: 2019-05-14 17:50:26.379000000~2019-05-14 18:05:27.471000000, Loss: 0.6888, Nodes_count: 25776, Cost Time: 2185.38s
Time: 2019-05-14 18:05:27.471000000~2019-05-14 18:20:29.235000000, Loss: 0.6603, Nodes_count: 27036, Cost Time: 2260.20s
Time: 2019-05-14 18:20:29.235000000~2019-05-14 18:35:31.058000000, Loss: 0.6790, Nodes_count: 27745, Cost Time: 2318.42s
Time: 2019-05-14 18:35:31.058000000~2019-05-14 18:50:32.147000000, Loss: 0.6484, Nodes_count: 28201, Cost Time: 2365.31s
Time: 2019-05-14 18:50:32.147000000~2019-05-14 19:05:39.105000000, Loss: 0.6455, Nodes_count: 28709, Cost Time: 2399.31s
Time: 2019-05-14 19:05:39.105000

In [19]:
ans_5_15=test_day_new(graph_5_15,"graph_5_15")

after merge: TemporalData(dst=[8379621], msg=[8379621, 42], src=[8379621], t=[8379621])
Time: 2019-05-15 00:00:01.169000000~2019-05-15 01:20:28.968000000, Loss: 2.0546, Nodes_count: 64, Cost Time: 0.05s
Time: 2019-05-15 01:20:28.968000000~2019-05-15 02:41:31.305000000, Loss: 1.2460, Nodes_count: 115, Cost Time: 0.11s
Time: 2019-05-15 02:41:31.305000000~2019-05-15 04:02:33.803000000, Loss: 0.8808, Nodes_count: 167, Cost Time: 0.17s
Time: 2019-05-15 04:02:33.803000000~2019-05-15 05:22:53.612000000, Loss: 1.3112, Nodes_count: 228, Cost Time: 0.25s
Time: 2019-05-15 05:22:53.612000000~2019-05-15 06:42:56.141000000, Loss: 1.5039, Nodes_count: 293, Cost Time: 0.31s
Time: 2019-05-15 06:42:56.141000000~2019-05-15 08:02:58.920000000, Loss: 1.0764, Nodes_count: 355, Cost Time: 0.38s
Time: 2019-05-15 08:02:58.920000000~2019-05-15 09:24:01.092000000, Loss: 1.3291, Nodes_count: 406, Cost Time: 0.44s
Time: 2019-05-15 09:24:01.092000000~2019-05-15 10:46:03.642000000, Loss: 1.0905, Nodes_count: 446, Co

In [20]:
ans_5_17=test_day_new(graph_5_17,"graph_5_17")

after merge: TemporalData(dst=[7520878], msg=[7520878, 42], src=[7520878], t=[7520878])
Time: 2019-05-17 00:00:00.162000000~2019-05-17 00:17:05.023000000, Loss: 0.5421, Nodes_count: 801, Cost Time: 15.61s
Time: 2019-05-17 00:17:05.023000000~2019-05-17 00:32:06.529000000, Loss: 0.5200, Nodes_count: 860, Cost Time: 20.06s
Time: 2019-05-17 00:32:06.529000000~2019-05-17 00:47:07.880000000, Loss: 0.4834, Nodes_count: 955, Cost Time: 41.46s
Time: 2019-05-17 00:47:07.880000000~2019-05-17 01:02:09.047000000, Loss: 0.4778, Nodes_count: 1003, Cost Time: 45.97s
Time: 2019-05-17 01:02:09.047000000~2019-05-17 01:17:10.410000000, Loss: 0.4721, Nodes_count: 1059, Cost Time: 67.84s
Time: 2019-05-17 01:17:10.410000000~2019-05-17 01:32:11.778000000, Loss: 0.5005, Nodes_count: 1111, Cost Time: 72.82s
Time: 2019-05-17 01:32:11.778000000~2019-05-17 01:47:12.976000000, Loss: 0.4731, Nodes_count: 1167, Cost Time: 94.07s
Time: 2019-05-17 01:47:12.976000000~2019-05-17 02:02:14.158000000, Loss: 0.4686, Nodes_co

Time: 2019-05-17 17:06:50.973000000~2019-05-17 17:21:51.831000000, Loss: 0.5506, Nodes_count: 21215, Cost Time: 1878.68s
Time: 2019-05-17 17:21:51.831000000~2019-05-17 17:36:53.217000000, Loss: 0.6196, Nodes_count: 21254, Cost Time: 1884.60s
Time: 2019-05-17 17:36:53.217000000~2019-05-17 17:51:54.220000000, Loss: 0.5513, Nodes_count: 21298, Cost Time: 1898.22s
Time: 2019-05-17 17:51:54.220000000~2019-05-17 18:06:55.854000000, Loss: 0.6136, Nodes_count: 21337, Cost Time: 1903.64s
Time: 2019-05-17 18:06:55.854000000~2019-05-17 18:21:57.246000000, Loss: 0.5378, Nodes_count: 21379, Cost Time: 1918.97s
Time: 2019-05-17 18:21:57.246000000~2019-05-17 18:36:58.618000000, Loss: 0.6231, Nodes_count: 21417, Cost Time: 1924.50s
Time: 2019-05-17 18:36:58.618000000~2019-05-17 18:52:00.027000000, Loss: 0.5331, Nodes_count: 21469, Cost Time: 1939.00s
Time: 2019-05-17 18:52:00.027000000~2019-05-17 19:07:01.518000000, Loss: 0.6418, Nodes_count: 21514, Cost Time: 1944.11s
Time: 2019-05-17 19:07:01.518000

# Initialize the node IDF

In [21]:
# share_node_IDF = mp.Manager().dict()

node_set=set()

file_list=[]


file_path="graph_5_9/"
file_l=os.listdir("graph_5_9/")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_5_11/"
file_l=os.listdir("graph_5_11/")
for i in file_l:
    file_list.append(file_path+i)


file_path="graph_5_12/"
file_l=os.listdir("graph_5_12/")
for i in file_l:
    file_list.append(file_path+i)

node_IDF={}
node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']):
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']):
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF    


torch.save(node_IDF,"node_IDF_5_9_12")
print("IDF weight calculate complete!")


100%|████████████████████████████████████████████████████████████████████████████████████████| 153/153 [02:32<00:00,  1.00it/s]

IDF weight calculate complete!


In [22]:
len(node_IDF)

24376

In [23]:
file_list_5_9_12 = file_list

In [24]:
def cal_train_IDF(find_str,file_list):
    include_count=0
    for f_path in (file_list):
        f=open(f_path)
        if find_str in f.read():
            include_count+=1             
    IDF=math.log(len(file_list)/(include_count+1))
    return IDF


def cal_IDF(find_str,file_path,file_list):
    file_list=os.listdir(file_path)
    include_count=0
    different_neighbor=set()
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_redundant(find_str,edge_list):
    
    different_neighbor=set()
    for e in edge_list:
        if find_str in str(e):
            different_neighbor.add(e[0])
            different_neighbor.add(e[1])
    return len(different_neighbor)-2

def cal_anomaly_loss(loss_list,edge_list,file_path):
    
    if len(loss_list)!=len(edge_list):
        print("error!")
        return 0
    count=0
    loss_sum=0
    loss_std=std(loss_list)
    loss_mean=mean(loss_list)
    edge_set=set()
    node_set=set()
    node2redundant={}
    
    thr=loss_mean+1.5*loss_std

    print("thr:",thr)

    for i in range(len(loss_list)):
        if loss_list[i]>thr:
            count+=1
            src_node=edge_list[i][0]
            dst_node=edge_list[i][1]
            
            loss_sum+=loss_list[i]
    
            node_set.add(src_node)
            node_set.add(dst_node)
            edge_set.add(edge_list[i][0]+edge_list[i][1])
    return count, loss_sum/(count + 0.000000001) ,node_set,edge_set
#     return count, count/len(loss_list)

# Construct the relations between time windows

In [25]:
# node_IDF=torch.load("node_IDF")


def cal_set_rel_bak(s1,s2,file_list):
    new_s=s1 & s2
    count=0
    for i in new_s:
#     jdata=json.loads(i)
        if 'netflow' not in i and '/data/system/' not in i \
            and '/storage/emulated/' not in i \
            and  '/data/data/com.android' not in i \
            and  '/proc/' not in i \
            and '/sys/devices/' not in i \
            and 'org.mozilla.fennec_vagrant' not in i \
            and 'mark.via.gp' not in i \
            and '/data/system_ce/' not in i \
            and '/Camera' not in i \
            and 'kohimovie.info.kohimovies' not in i \
            and '.dziauz.tinyflashlight' not in i \
            and 'com.' not in i \
            and 'android.process.media' not in i \
            and 'temp-index' not in i \
            and '/dev/binder' not in i \
            and 'vanilla' not in i:                       
        
        #             and 'screencap' not in i \
        
#         and '.dziauz.tinyflashlight' not in i \
#             and '/data/system_ce/ not in i \
            
#         and 'usr' not in i and 'proc' not in i and '675' not in i and 'firefox' not in i and 'tmp' not in i and 'thunderbird' not in i
#         'netflow' not in i
#         and 'usr' not in i and 'var' not in i
            if i in node_IDF.keys():
                IDF=node_IDF[i]
            else:
                IDF=math.log(len(file_list)/(1))           
                   
#             print(IDF)
#             print(len(file_list))
            if IDF>math.log(len(file_list)*0.9/(1))  :
#             if IDF>4.7:
                print("node:",i," IDF:",IDF)
                count+=1
    return count

# label generation

In [26]:
labels={}
pred_label={}
    
filelist = os.listdir("graph_5_14")
for f in filelist:
    labels["graph_5_14/"+f]=0
    pred_label["graph_5_14/"+f]=0

filelist = os.listdir("graph_5_15")
for f in filelist:
    labels["graph_5_15/"+f]=0
    pred_label["graph_5_15/"+f]=0
    
filelist = os.listdir("graph_5_17")
for f in filelist:
    labels["graph_5_17/"+f]=0
    pred_label["graph_5_17/"+f]=0

In [27]:
attack_list=[    
    'graph_5_15/2019-05-15 14:07:59.753000000~2019-05-15 14:23:00.265000000.txt',
    'graph_5_15/2019-05-15 14:23:00.265000000~2019-05-15 14:38:02.135000000.txt',
    'graph_5_15/2019-05-15 15:38:59.175000000~2019-05-15 15:55:38.955000000.txt', 
    'graph_5_15/2019-05-15 15:55:38.955000000~2019-05-15 16:11:27.687000000.txt',    

    'graph_5_17/2019-05-17 14:50:52.897000000~2019-05-17 15:06:00.371000000.txt', 
    'graph_5_17/2019-05-17 15:06:00.371000000~2019-05-17 15:21:40.474000000.txt', 
    'graph_5_17/2019-05-17 15:21:40.474000000~2019-05-17 15:36:41.964000000.txt', 
    #     'graph_5_17/2019-05-17 15:36:41.964000000~2019-05-17 15:51:43.493000000.txt',
    'graph_5_17/2019-05-17 15:51:43.493000000~2019-05-17 16:06:44.953000000.txt', 
    'graph_5_17/2019-05-17 16:21:46.485000000~2019-05-17 16:36:47.843000000.txt', 
    'graph_5_17/2019-05-17 16:36:47.843000000~2019-05-17 16:51:49.352000000.txt'

]

# graph_5_17/2019-05-17 11:48:07.561000000~2019-05-17 12:03:08.991000000.txt  # might be anomalous
for i in labels:
    if i in attack_list:
        labels[i]=1
    else:
        labels[i]=0

# for i in attack_list:
#     labels[i]=1

In [28]:
labels

{'graph_5_14/2019-05-14 12:02:56.496000000~2019-05-14 12:17:57.806000000.txt': 0,
 'graph_5_14/2019-05-14 21:19:21.805000000~2019-05-14 22:39:40.614000000.txt': 0,
 'graph_5_14/2019-05-14 15:20:00.233000000~2019-05-14 15:35:02.947000000.txt': 0,
 'graph_5_14/2019-05-14 09:17:42.517000000~2019-05-14 09:32:44.093000000.txt': 0,
 'graph_5_14/2019-05-14 15:50:03.958000000~2019-05-14 16:05:05.450000000.txt': 0,
 'graph_5_14/2019-05-14 01:01:59.206000000~2019-05-14 01:17:00.513000000.txt': 0,
 'graph_5_14/2019-05-14 08:47:39.890000000~2019-05-14 09:02:41.157000000.txt': 0,
 'graph_5_14/2019-05-14 11:47:54.952000000~2019-05-14 12:02:56.496000000.txt': 0,
 'graph_5_14/2019-05-14 06:32:28.279000000~2019-05-14 06:47:29.637000000.txt': 0,
 'graph_5_14/2019-05-14 04:32:17.569000000~2019-05-14 04:47:18.740000000.txt': 0,
 'graph_5_14/2019-05-14 08:17:37.204000000~2019-05-14 08:32:38.759000000.txt': 0,
 'graph_5_14/2019-05-14 07:17:32.480000000~2019-05-14 07:32:33.922000000.txt': 0,
 'graph_5_14/201

# 5-14

In [29]:
# node_IDF=torch.load("node_IDF")
y_data_5_14=[]
df_list_5_14=[]
# node_set_list=[]
history_list_5_14=[]
tw_que=[]
his_tw={}
current_tw={}

file_path_list=[]

file_path="graph_5_14/"
file_l=os.listdir("graph_5_14/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_14.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_5_14/")

    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_5_14:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list_5_9_12)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_14.append(temp_hq)
    index_count+=1
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 1.2994436352680885
graph_5_14/2019-05-14 00:00:00.110000000~2019-05-14 00:16:55.164000000.txt    2.3240000991915886  count: 1569  percentage: 0.0383056640625  node count: 409  edge count: 408
index_count: 1
thr: 1.4782385488367134
graph_5_14/2019-05-14 00:16:55.164000000~2019-05-14 00:31:56.512000000.txt    1.8541332864371292  count: 1270  percentage: 0.09540264423076923  node count: 32  edge count: 35
index_count: 2
thr: 1.0407567938091484
graph_5_14/2019-05-14 00:31:56.512000000~2019-05-14 00:46:57.832000000.txt    1.469677932960212  count: 835  percentage: 0.020908453525641024  node count: 106  edge count: 104
index_count: 3
thr: 1.5060143560294126
graph_5_14/2019-05-14 00:46:57.832000000~2019-05-14 01:01:59.206000000.txt    2.199582764401979  count: 735  percentage: 0.059814453125  node count: 29  edge count: 28
index_count: 4
thr: 1.0218455520756344
graph_5_14/2019-05-14 01:01:59.206000000~2019-05-14 01:17:00.513000000.txt    1.2468562588354621  count: 815  per

thr: 1.5154519355211842
graph_5_14/2019-05-14 09:47:44.971000000~2019-05-14 10:02:46.433000000.txt    1.68470143776757  count: 11606  percentage: 0.03976836622807017  node count: 528  edge count: 542
index_count: 40
thr: 1.4950500794685122
graph_5_14/2019-05-14 10:02:46.433000000~2019-05-14 10:17:47.485000000.txt    1.7302317450874771  count: 16468  percentage: 0.04814979416167665  node count: 596  edge count: 607
index_count: 41
thr: 1.499693620907286
graph_5_14/2019-05-14 10:17:47.485000000~2019-05-14 10:32:48.904000000.txt    1.7762107164568588  count: 5637  percentage: 0.041390096334586464  node count: 485  edge count: 492
index_count: 42
thr: 1.4340534199578903
graph_5_14/2019-05-14 10:32:48.904000000~2019-05-14 10:47:49.773000000.txt    1.714631811360223  count: 8303  percentage: 0.04884577371987952  node count: 457  edge count: 468
index_count: 43
thr: 1.5123144511343576
graph_5_14/2019-05-14 10:47:49.773000000~2019-05-14 11:02:51.213000000.txt    1.6992958948041406  count: 1349

In [30]:
name_list=[]
for hl in history_list_5_14:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

# 5-15

In [31]:
# node_IDF=torch.load("node_IDF")
y_data_5_15=[]
df_list_5_15=[]
# node_set_list=[]
history_list_5_15=[]
tw_que=[]
his_tw={}
current_tw={}
loss_list_5_15=[]

file_path_list=[]

file_path="graph_5_15/"
file_l=os.listdir("graph_5_15/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_15.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_5_15/")
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_5_15:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list_5_9_12)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_15.append(temp_hq)
    index_count+=1
    loss_list_5_15.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 3.1683880261853683
graph_5_15/2019-05-15 00:00:01.169000000~2019-05-15 01:20:28.968000000.txt    0.0  count: 0  percentage: 0.0  node count: 0  edge count: 0
index_count: 1
thr: 2.087508243346082
graph_5_15/2019-05-15 01:20:28.968000000~2019-05-15 02:41:31.305000000.txt    0.0  count: 0  percentage: 0.0  node count: 0  edge count: 0
index_count: 2
thr: 1.6156690263165976
graph_5_15/2019-05-15 02:41:31.305000000~2019-05-15 04:02:33.803000000.txt    0.0  count: 0  percentage: 0.0  node count: 0  edge count: 0
index_count: 3
thr: 2.40459458662835
graph_5_15/2019-05-15 04:02:33.803000000~2019-05-15 05:22:53.612000000.txt    2.5836529671984367  count: 160  percentage: 0.15625  node count: 2  edge count: 1
index_count: 4
thr: 2.484223075000419
graph_5_15/2019-05-15 05:22:53.612000000~2019-05-15 06:42:56.141000000.txt    0.0  count: 0  percentage: 0.0  node count: 0  edge count: 0
index_count: 5
thr: 1.8504494380717709
graph_5_15/2019-05-15 06:42:56.141000000~2019-05-15 08

thr: 1.537049552112133
graph_5_15/2019-05-15 18:29:15.893000000~2019-05-15 18:44:17.260000000.txt    1.7855619591379732  count: 14272  percentage: 0.04756825938566553  node count: 872  edge count: 904
index_count: 35
thr: 1.4427678158666049
graph_5_15/2019-05-15 18:44:17.260000000~2019-05-15 18:59:18.387000000.txt    1.6951824174970394  count: 5826  percentage: 0.043430939885496185  node count: 281  edge count: 285
index_count: 36
thr: 1.5122281101908
graph_5_15/2019-05-15 18:59:18.387000000~2019-05-15 19:14:19.745000000.txt    1.7272879987785017  count: 14120  percentage: 0.04203982469512195  node count: 625  edge count: 636
index_count: 37
thr: 1.4999088661364512
graph_5_15/2019-05-15 19:14:19.745000000~2019-05-15 19:29:20.800000000.txt    1.6921135548526869  count: 16685  percentage: 0.04476358602335165  node count: 503  edge count: 503
index_count: 38
thr: 1.522944163388003
graph_5_15/2019-05-15 19:29:20.800000000~2019-05-15 19:44:22.319000000.txt    1.731481507383075  count: 13967

In [32]:

name_list=[]
for hl in history_list_5_15:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

['graph_5_15/2019-05-15 10:46:03.642000000~2019-05-15 11:06:20.806000000.txt', 'graph_5_15/2019-05-15 14:07:59.753000000~2019-05-15 14:23:00.265000000.txt', 'graph_5_15/2019-05-15 14:23:00.265000000~2019-05-15 14:38:02.135000000.txt', 'graph_5_15/2019-05-15 15:38:59.175000000~2019-05-15 15:55:38.955000000.txt', 'graph_5_15/2019-05-15 15:55:38.955000000~2019-05-15 16:11:27.687000000.txt', 'graph_5_15/2019-05-15 16:11:27.687000000~2019-05-15 16:26:39.781000000.txt']
873.3464440179548


# 5-17

In [33]:
# node_IDF=torch.load("node_IDF")
y_data_5_17=[]
df_list_5_17=[]
# node_set_list=[]
history_list_5_17=[]
tw_que=[]
his_tw={}
current_tw={}

loss_list_5_17=[]

file_path_list=[]

file_path="graph_5_17/"
file_l=os.listdir("graph_5_17/")
for i in file_l:
    file_path_list.append(file_path+i)

index_count=0
for f_path in sorted(file_path_list):
    f=open(f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_5_17.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_5_17/")
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list_5_17:
        for his_tw in hq:
            if cal_set_rel_bak(current_tw['nodeset'],his_tw['nodeset'],file_list_5_9_12)!=0 and current_tw['name']!=his_tw['name']:
                print("history queue:",his_tw['name'])
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list_5_17.append(temp_hq)
    index_count+=1
    loss_list_5_17.append(loss_avg)
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))

index_count: 0
thr: 1.2396995421672876
graph_5_17/2019-05-17 00:00:00.162000000~2019-05-17 00:17:05.023000000.txt    2.2705906906490316  count: 990  percentage: 0.031186995967741934  node count: 282  edge count: 291
index_count: 1
thr: 1.1348125571401906
graph_5_17/2019-05-17 00:17:05.023000000~2019-05-17 00:32:06.529000000.txt    1.667140552249508  count: 305  percentage: 0.0372314453125  node count: 15  edge count: 18
index_count: 2
thr: 1.0345511625831574
graph_5_17/2019-05-17 00:32:06.529000000~2019-05-17 00:47:07.880000000.txt    1.961928488204487  count: 344  percentage: 0.010179924242424242  node count: 61  edge count: 62
index_count: 3
thr: 1.0156902668151262
graph_5_17/2019-05-17 00:47:07.880000000~2019-05-17 01:02:09.047000000.txt    1.4832856792555933  count: 130  percentage: 0.015869140625  node count: 6  edge count: 5
index_count: 4
thr: 0.981017382587746
graph_5_17/2019-05-17 01:02:09.047000000~2019-05-17 01:17:10.410000000.txt    1.4612201134716314  count: 184  percentag

thr: 1.536448356409168
graph_5_17/2019-05-17 09:47:52.897000000~2019-05-17 10:02:57.226000000.txt    1.7534287099185661  count: 12653  percentage: 0.05191783744747899  node count: 473  edge count: 481
index_count: 40
thr: 1.5078394245768911
graph_5_17/2019-05-17 10:02:57.226000000~2019-05-17 10:17:59.598000000.txt    1.7567328132033668  count: 15830  percentage: 0.05051955678104575  node count: 831  edge count: 860
index_count: 41
thr: 1.5185919554989606
graph_5_17/2019-05-17 10:17:59.598000000~2019-05-17 10:33:01.000000000.txt    1.6581166637192228  count: 22392  percentage: 0.042133309248554913  node count: 644  edge count: 648
index_count: 42
thr: 1.4496993737458292
graph_5_17/2019-05-17 10:33:01.000000000~2019-05-17 10:48:02.004000000.txt    1.6515785491940222  count: 10132  percentage: 0.043781111725663714  node count: 236  edge count: 235
index_count: 43
thr: 1.5339011066633872
graph_5_17/2019-05-17 10:48:02.004000000~2019-05-17 11:03:03.339000000.txt    1.7268913049731953  count

thr: 1.2617493156536603
graph_5_17/2019-05-17 17:36:53.217000000~2019-05-17 17:51:54.220000000.txt    2.4691279896931744  count: 649  percentage: 0.02880859375  node count: 31  edge count: 28
index_count: 71
thr: 1.3948741515516274
graph_5_17/2019-05-17 17:51:54.220000000~2019-05-17 18:06:55.854000000.txt    2.329598121522706  count: 574  percentage: 0.050958806818181816  node count: 29  edge count: 26
index_count: 72
thr: 1.2257049771073185
graph_5_17/2019-05-17 18:06:55.854000000~2019-05-17 18:21:57.246000000.txt    2.6283194073036733  count: 537  percentage: 0.0218505859375  node count: 26  edge count: 22
index_count: 73
thr: 1.430576042023203
graph_5_17/2019-05-17 18:21:57.246000000~2019-05-17 18:36:58.618000000.txt    2.453868646388667  count: 563  percentage: 0.049982244318181816  node count: 26  edge count: 25
index_count: 74
thr: 1.2142718307793983
graph_5_17/2019-05-17 18:36:58.618000000~2019-05-17 18:52:00.027000000.txt    2.609119129039968  count: 526  percentage: 0.02140299

In [34]:
name_list=[]
for hl in history_list_5_17:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
#     name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name']) 
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

['graph_5_17/2019-05-17 00:32:06.529000000~2019-05-17 00:47:07.880000000.txt', 'graph_5_17/2019-05-17 11:33:05.441000000~2019-05-17 11:48:07.561000000.txt', 'graph_5_17/2019-05-17 14:50:52.897000000~2019-05-17 15:06:00.371000000.txt', 'graph_5_17/2019-05-17 15:06:00.371000000~2019-05-17 15:21:40.474000000.txt', 'graph_5_17/2019-05-17 15:21:40.474000000~2019-05-17 15:36:41.964000000.txt', 'graph_5_17/2019-05-17 15:51:43.493000000~2019-05-17 16:06:44.953000000.txt', 'graph_5_17/2019-05-17 16:21:46.485000000~2019-05-17 16:36:47.843000000.txt', 'graph_5_17/2019-05-17 16:36:47.843000000~2019-05-17 16:51:49.352000000.txt']
7193.30948885869


In [35]:
from sklearn.metrics import average_precision_score, roc_auc_score

# from sklearn.metrics import plot_roc_curve,roc_curve,auc,roc_auc_score
import torch
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

def plot_thr():
    np.seterr(invalid='ignore')
    step=0.01
    thr_list=torch.arange(-5,5,step)
    
    

    precision_list=[]
    recall_list=[]
    fscore_list=[]
    accuracy_list=[]
    auc_val_list=[]
    for thr in thr_list:
        threshold=thr
        y_prediction=[]
        for i in y_test_scores:
            if i >threshold:
                y_prediction.append(1)
            else:
                y_prediction.append(0)
        precision,recall,fscore,accuracy,auc_val=classifier_evaluation(y_test, y_prediction)   
        precision_list.append(float(precision))
        recall_list.append(float(recall))
        fscore_list.append(float(fscore))
        accuracy_list.append(float(accuracy))
        auc_val_list.append(float(auc_val))

    max_fscore=max(fscore_list)
    max_fscore_index=fscore_list.index(max_fscore)
    print(max_fscore_index)
    print("max threshold:",thr_list[max_fscore_index])
    print('precision:',precision_list[max_fscore_index])
    print('recall:',recall_list[max_fscore_index])
    print('fscore:',fscore_list[max_fscore_index])
    print('accuracy:',accuracy_list[max_fscore_index])    
    print('auc:',auc_val_list[max_fscore_index])

    plt.plot(thr_list,precision_list,color='red',label='precision',linewidth=2.0,linestyle='-')
    plt.plot(thr_list,recall_list,color='orange',label='recall',linewidth=2.0,linestyle='solid')
    plt.plot(thr_list,fscore_list,color='y',label='F-score',linewidth=2.0,linestyle='dashed')
    plt.plot(thr_list,accuracy_list,color='g',label='accuracy',linewidth=2.0,linestyle='dashdot')
    plt.plot(thr_list,auc_val_list,color='b',label='auc_val',linewidth=2.0,linestyle='dotted')


    plt.xlabel("Threshold", fontdict={'size': 16})
    plt.ylabel("Rate", fontdict={'size': 16})
    plt.title("Different evaluation Indicators by varying threshold value", fontdict={'size': 12})
    plt.legend(loc='best', fontsize=12, markerscale=0.5)
    plt.show()

def classifier_evaluation(y_test, y_test_pred):
    # groundtruth, pred_value
    tn, fp, fn, tp =confusion_matrix(y_test, y_test_pred).ravel()
#     tn+=100
#     print(clf_name," : ")
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    fscore=2*(precision*recall)/(precision+recall)    
    auc_val=roc_auc_score(y_test, y_test_pred)
    print("precision:",precision)
    print("recall:",recall)
    print("fscore:",fscore)
    print("accuracy:",accuracy)
    print("auc_val:",auc_val)
    return precision,recall,fscore,accuracy,auc_val

def minmax(data):
    min_val=min(data)
    max_val=max(data)
    ans=[]
    for i in data:
        ans.append((i-min_val)/(max_val-min_val))
    return ans





In [36]:
y=[]
y_pred=[]
for i in labels:
    y.append(labels[i])
    y_pred.append(pred_label[i])

In [37]:
classifier_evaluation(y,y_pred)

tn: 218
fp: 4
fn: 0
tp: 10
precision: 0.7142857142857143
recall: 1.0
fscore: 0.8333333333333333
accuracy: 0.9827586206896551
auc_val: 0.990990990990991


(0.7142857142857143,
 1.0,
 0.8333333333333333,
 0.9827586206896551,
 0.990990990990991)

# count the number of the attack edges

In [38]:
def keyword_hit(line):
    attack_nodes=[
            'barephone-instr.apk',
        'screencap-instr.apk',
           'de.belu.appstarter',
        './run_webserver.sh',
        'appstarter-instr.apk',
        'screenshot.png',
        'screenshot',
        '/dev/msm_g711tlaw',
        'com.android.providers.contacts',
        'barephone',
        'busybox',

        
        '/data/local/tmp',
        'calllog.db',
        'calendar.db',        
        'external.db',
        'internal.db',
        'lastAccess.db',
        'mmssms.db',
        

#         '77.138.117.150',      
#         '128.55.12.33',
#         '128.55.12.233',
#         '128.55.12.166',
#         '49.8.46.240',
#         '42.183.7.162',
#         '133.39.25.45', 
        ]

    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag



files=[]

filelist = os.listdir("graph_5_15")
for f in filelist:    
    files.append("graph_5_15/"+f) 
    
filelist = os.listdir("graph_5_17")
for f in filelist:
    files.append("graph_5_17/"+f) 

In [39]:
attack_edge_count=0
for fpath in tqdm(files):
    f=open(fpath)
    for line in f:
        if keyword_hit(line):
            attack_edge_count+=1
print(attack_edge_count)

100%|████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:17<00:00,  8.45it/s]

4044


# Visualization

In [5]:
import os

from graphviz import Digraph
import networkx as nx
import datetime
import community.community_louvain as community_louvain
from tqdm import tqdm



# Some common path abstraction for visualization
replace_dic={
    '/run/shm/':'/run/shm/*',
#     '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/':'/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/*',
   '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
    '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',    
    '/data/replay_logdb/':'/data/replay_logdb/*', 
    '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*', 
    
    '/usr/share/applications/':'/usr/share/applications/*', 
    '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',     
    '/proc/':'/proc/*', 
     '/stat':'*/stat', 
    '/etc/bash_completion.d/':'/etc/bash_completion.d/*', 
    '/usr/bin/python2.7':'/usr/bin/python2.7/*', 
     '/usr/lib/python2.7':'/usr/lib/python2.7/*', 
'/data/data/org.mozilla.fennec_firefox_dev/cache/':'/data/data/org.mozilla.fennec_firefox_dev/cache/*',
    'UNNAMED':'UNNAMED *',
}


def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name


# Users should manually put the detected anomalous time windows here
attack_list = [
    'graph_5_15/2019-05-15 14:07:59.753000000~2019-05-15 14:23:00.265000000.txt',
    'graph_5_15/2019-05-15 14:23:00.265000000~2019-05-15 14:38:02.135000000.txt',
    'graph_5_15/2019-05-15 15:38:59.175000000~2019-05-15 15:55:38.955000000.txt', 
    'graph_5_15/2019-05-15 15:55:38.955000000~2019-05-15 16:11:27.687000000.txt',    

    'graph_5_17/2019-05-17 14:50:52.897000000~2019-05-17 15:06:00.371000000.txt', 
    'graph_5_17/2019-05-17 15:06:00.371000000~2019-05-17 15:21:40.474000000.txt', 
    'graph_5_17/2019-05-17 15:21:40.474000000~2019-05-17 15:36:41.964000000.txt', 
    'graph_5_17/2019-05-17 15:51:43.493000000~2019-05-17 16:06:44.953000000.txt', 
    'graph_5_17/2019-05-17 16:21:46.485000000~2019-05-17 16:36:47.843000000.txt', 
    'graph_5_17/2019-05-17 16:36:47.843000000~2019-05-17 16:51:49.352000000.txt'
]

original_edges_count = 0
graphs = []
gg = nx.DiGraph()
count = 0
for path in tqdm(attack_list):
    if ".txt" in path:
        line_count = 0
        node_set = set()
        tempg = nx.DiGraph()
        f = open(path, "r")
        edge_list = []
        for line in f:
            count += 1
            l = line.strip()
            jdata = eval(l)
            edge_list.append(jdata)

        edge_list = sorted(edge_list, key=lambda x: x['loss'], reverse=True)
        original_edges_count += len(edge_list)

        loss_list = []
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean = mean(loss_list)
        loss_std = std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr = loss_mean + 1.5 * loss_std
        print("thr:", thr)
        for e in edge_list:
            if e['loss'] > thr:
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),
                               str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))), str(hashgen(replace_path_name(e['dstmsg']))),
                            loss=e['loss'], srcmsg=e['srcmsg'], dstmsg=e['dstmsg'], edge_type=e['edge_type'],
                            time=e['time'])


partition = community_louvain.best_partition(gg.to_undirected())

# Generate the candidate subgraphs based on community discovery results
communities = {}
max_partition = 0
for i in partition:
    if partition[i] > max_partition:
        max_partition = partition[i]
for i in range(max_partition + 1):
    communities[i] = nx.DiGraph()
for e in gg.edges:
    communities[partition[e[0]]].add_edge(e[0], e[1])
    communities[partition[e[1]]].add_edge(e[0], e[1])


# Define the attack nodes. They are **only be used to plot the colors of attack nodes and edges**.
# They won't change the detection results.
def attack_edge_flag(msg):
    attack_edge_type=[
    'barephone-instr.apk',
        'screencap-instr.apk',
           'de.belu.appstarter',
        './run_webserver.sh',
        'appstarter-instr.apk',
        'screenshot.png',
        'screenshot',
        '/dev/msm_g711tlaw',
        'com.android.providers.contacts',
        'barephone',
        'busybox',
        'screencap',
 
        '/data/local/tmp',
        'calllog.db',
        'calendar.db',        
        'external.db',
        'internal.db',
        'lastAccess.db',
        'mmssms.db',
    ]
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
    return flag


# Plot and render candidate subgraph
os.system(f"mkdir -p ./graph_visual/")
graph_index = 0
for c in communities:
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'

    for e in communities[c].edges:
        try:
            temp_edge = gg.edges[e]
            srcnode = e['srcnode']
            dstnode = e['dstnode']
        except:
            pass

        if True:
            # source node
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape = 'box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape = 'oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape = 'diamond'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color = 'red'
            else:
                src_node_color = 'blue'
            dot.node(name=str(hashgen(replace_path_name(temp_edge['srcmsg']))), label=str(
                replace_path_name(temp_edge['srcmsg']) + str(
                    partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), color=src_node_color,
                     shape=src_shape)

            # destination node
            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape = 'box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape = 'oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape = 'diamond'
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color = 'red'
            else:
                dst_node_color = 'blue'
            dot.node(name=str(hashgen(replace_path_name(temp_edge['dstmsg']))), label=str(
                replace_path_name(temp_edge['dstmsg']) + str(
                    partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), color=dst_node_color,
                     shape=dst_shape)

            if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
                edge_color = 'red'
            else:
                edge_color = 'blue'
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),
                     str(hashgen(replace_path_name(temp_edge['dstmsg']))), label=temp_edge['edge_type'],
                     color=edge_color)

    dot.render(f'./graph_visual/subgraph_' + str(graph_index), view=False)
    graph_index += 1

 10%|█████████                                                                                  | 1/10 [00:01<00:09,  1.10s/it]

0.659518992846858
0.5674487554266885
thr: 1.5106921259868908
0.5534007007233754
0.4498819074684684
thr: 1.228223561926078


 30%|███████████████████████████▎                                                               | 3/10 [00:02<00:06,  1.16it/s]

0.6576207372757144
0.5560456364225123
thr: 1.4916891919094828
0.5148039244530523
0.38085547794764996
thr: 1.0860871413745272


 50%|█████████████████████████████████████████████▌                                             | 5/10 [00:04<00:04,  1.15it/s]

...

0.6246477632806717
0.5409780570858713
thr: 1.4361148489094786
0.7208051666068069
0.5799459685436368
thr: 1.590724119422262


 90%|█████████████████████████████████████████████████████████████████████████████████▉         | 9/10 [00:04<00:00,  2.98it/s]

0.7665564077627399
0.5961178147288376
thr: 1.6607331298559962
0.7082760480609182
0.5570571217512844
thr: 1.5438617306878446


100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.93it/s]

0.5543714374887345
0.4757021005456081
thr: 1.2679245883071466
